This notebook shows how to output pivot table such that the number intervals of row/columns are properly sorted. 

In [14]:
import pandas as pd
import numpy as np

In [85]:
df = pd.DataFrame(data={'x':np.random.randint(0,10000,1000), 
                        'y':np.random.randint(0,1000,1000),
                        'z':[random.choice(['a','b','c']) for i in range(1000)]})

In [86]:
mybin_1 = [0, 10, 20, 50, 1e+2, 2e+2, 5e+2, 1e+3, 2e+3, 5e+3, 1e+4, 2e+4, 5e+4]
mybin_2 = [0, 10, 20, 1e+2, 2e+2, 1e+3, 1e+4, 5e+4]

In [87]:
df['x_bin'] = pd.cut(x=df.x, bins=mybin_1)
df['y_bin'] = pd.cut(x=df.y, bins=mybin_2)

In [88]:
df[:10]

,x,y,z,x_bin,y_bin
0,5358,685,a,"(5000, 10000]","(200, 1000]"
1,8310,929,a,"(5000, 10000]","(200, 1000]"
2,5575,553,c,"(5000, 10000]","(200, 1000]"
3,2471,7,a,"(2000, 5000]","(0, 10]"
4,7463,240,a,"(5000, 10000]","(200, 1000]"
5,5431,753,a,"(5000, 10000]","(200, 1000]"
6,2919,662,b,"(2000, 5000]","(200, 1000]"
7,8799,492,c,"(5000, 10000]","(200, 1000]"
8,5333,560,c,"(5000, 10000]","(200, 1000]"
9,436,412,b,"(200, 500]","(200, 1000]"


This is how you normally use crosstab, the row and column order is based on string-order.

In [89]:
pd.crosstab(df.x_bin, df.y_bin)

y_bin,"(0, 10]","(10, 20]","(100, 200]","(1000, 10000]","(10000, 50000]","(20, 100]","(200, 1000]"
x_bin,,,,,,,
"(0, 10]",0,0,0,0,0,0,1
"(10, 20]",0,0,0,0,0,0,0
"(100, 200]",0,0,2,0,0,0,8
"(1000, 2000]",1,0,9,0,0,10,82
"(10000, 20000]",0,0,0,0,0,0,0
"(20, 50]",0,0,0,0,0,0,6
"(200, 500]",0,0,5,0,0,3,23
"(2000, 5000]",5,2,31,0,0,38,220
"(20000, 50000]",0,0,0,0,0,0,0


This is how to make the rder looks good.

In [122]:
def relabel(myitem, right_most='inf'):
    """
    This function re-label the row/column of a data frame.
    Especially, the one that comes from pd.cut.
    [0, 1, 2, 3] --> ['[0,1)', '[1,2)', '[2,3)', '[3,right_most)']
    """
    new_item = ['']*len(myitem)
    for i in np.arange(len(myitem)-1):
        new_item[i] = "[%d,%d)" % (myitem[i], myitem[i+1])

    i = len(myitem)-1    
    new_item[i] = "[%d,%s)" % (myitem[i], right_most)
    return new_item


def crosstab2(X1, X2, mybin1=None, mybin2=None):
    """
    Produce pivolt table similar to crosstab except that the order of row or column is by the interval.
    X1 and X2: TimeSeries.
    mybin1 and mybin2: list of binning boundary.
    
    kittipat@
    June 18, 2015
    """
    
    if mybin1:
        bin1 = pd.cut(x=X1, bins=mybin1, labels=[s for s in mybin1[:-1]])
    else:
        bin1 = X1
        
    if mybin2:
        bin2 = pd.cut(x=X2, bins=mybin2, labels=[s for s in mybin2[:-1]])
    else:
        bin2 = X2
        
    df_pivot = pd.crosstab(bin1, bin2)
    
    if mybin1:
        df_pivot.index = relabel(df_pivot.index, str(mybin1[-1]))
    
    if mybin2:
        df_pivot.columns = relabel(df_pivot.columns, str(mybin2[-1])) 
    
    return df_pivot

In [124]:
crosstab2(df.x, df.y, mybin_1, mybin_2)

,"[0,10)","[10,20)","[20,100)","[100,200)","[200,1000)","[1000,10000)","[10000,50000.0)"
"[0,10)",0,0,0,0,1,0,0
"[10,20)",0,0,0,0,0,0,0
"[20,50)",0,0,0,0,6,0,0
"[50,100)",0,0,0,0,1,0,0
"[100,200)",0,0,0,2,8,0,0
"[200,500)",0,0,3,5,23,0,0
"[500,1000)",2,0,6,4,54,0,0
"[1000,2000)",1,0,10,9,82,0,0
"[2000,5000)",5,2,38,31,220,0,0
"[5000,10000)",7,0,49,51,376,0,0


In [126]:
crosstab2(df.x, df.z, mybin_1)

z,a,b,c
"[0,10)",1,0,0
"[10,20)",0,0,0
"[20,50)",3,2,1
"[50,100)",1,0,0
"[100,200)",0,5,5
"[200,500)",7,7,17
"[500,1000)",26,22,18
"[1000,2000)",35,34,34
"[2000,5000)",103,102,94
"[5000,10000)",177,140,166


In [127]:
crosstab2(df.z, df.x, [], mybin_1)

,"[0,10)","[10,20)","[20,50)","[50,100)","[100,200)","[200,500)","[500,1000)","[1000,2000)","[2000,5000)","[5000,10000)","[10000,20000)","[20000,50000.0)"
z,,,,,,,,,,,,
a,1,0,3,1,0,7,26,35,103,177,0,0
b,0,0,2,0,5,7,22,34,102,140,0,0
c,0,0,1,0,5,17,18,34,94,166,0,0
